In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml

# part 1

In [54]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))

In [71]:
neigh = pd.DataFrame(df[0])
neigh = neigh[neigh.Borough != 'Not assigned']
neigh.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [86]:
neigh = neigh.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(list)
neigh = neigh.sample(frac = 1).reset_index()
neigh['Neighborhood'] = neigh['Neighborhood'].str.join(', ')
neigh['Neighborhood'] = neigh['Neighborhood'].str.replace(' /', ',')
neigh.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
2,M4N,Central Toronto,Lawrence Park
3,M2H,North York,Hillcrest Village
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [72]:
neigh.shape

(103, 3)

# part 2

In [78]:
import geocoder
#postal_code = 'M4A'
#lat_lng_coords = None
#while(lat_lng_coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#    lat_lng_coords = g.latlng
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [85]:
map_df = pd.read_csv('http://cocl.us/Geospatial_data')
map_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [89]:
neigh_map = neigh.set_index('Postal code').join(map_df.set_index('Postal Code'))

In [90]:
neigh_map

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
M2H,North York,Hillcrest Village,43.803762,-79.363452
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...
M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
